In [6]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split


In [7]:
# !pip install pandas_datareader

In [11]:
#from pandas_datareader import data, wb
import pandas as pd
# the line below is the fix for is_list_like lub
pd.core.common.is_list_like = pd.api.types.is_list_like

import pandas_datareader as web
import datetime
start = datetime.datetime(2010, 1, 1)
end = datetime.datetime(2020, 1, 30)
stockList = ["PYPL", "CTSH", "TSLA"]
# data.head(-10)

In [12]:
# data.head(10)

In [13]:
result = dict()

for stock in stockList:
    data = web.DataReader(stock,'yahoo', start, end)
    data.columns = [x.strip() for x in data.columns]
    data.columns = [x.replace(' ', '_') for x in data.columns]
    
    efficiency = 1.0
    number = 0
    
    print("Finding how long in future, the model can predict with prediction score > 70% for stock: ",stock)
    
    while efficiency > 0.70:
        number += 1
        pd.concat([data.Close, data.Close.shift(-number)], axis=1)
        def valid(col):
            return "Close" not in col
        testData = data.dropna()

        # we remove the last observation (last row) because there is no tomorrow for the last row to predict!
        X = testData[[x for x in testData.columns if valid(x)]].iloc[0:-number]

        # for the dependent variable, we remove the NA, so that removes the last row, too
        y = testData.Close.shift(-number).dropna()
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
            
        # Create a model 
        rf_model = RandomForestRegressor()

        # Train the model
        rf_model.fit(X_train, y_train)
        efficiency = rf_model.score(X_test, y_test)
        if number%10 == 0:
            print("For",number,"\bth day in future, prediction score is", round(efficiency*100,2),"\b% for stock",stock)
    
    result.update({stock:number-1})

print("\n\nResult:")
print("Our model can predict with prediction score > 70% as shown below for each stock")
for stock in result:
    print(stock+":",result[stock])

Finding how long in future, the model can predict with prediction score > 70% for stock:  PYPL
For 10th day in future, prediction score is 98.73% for stock PYPL
For 20th day in future, prediction score is 98.08% for stock PYPL
For 30th day in future, prediction score is 97.64% for stock PYPL
For 40th day in future, prediction score is 97.19% for stock PYPL
For 50th day in future, prediction score is 96.46% for stock PYPL
For 60th day in future, prediction score is 96.97% for stock PYPL
For 70th day in future, prediction score is 96.2% for stock PYPL
For 80th day in future, prediction score is 95.8% for stock PYPL
For 90th day in future, prediction score is 94.58% for stock PYPL
For 100th day in future, prediction score is 95.4% for stock PYPL
For 110th day in future, prediction score is 94.67% for stock PYPL
For 120th day in future, prediction score is 93.93% for stock PYPL
For 130th day in future, prediction score is 94.05% for stock PYPL
For 140th day in future, prediction score is 9

For 240th day in future, prediction score is 74.6% for stock TSLA
For 250th day in future, prediction score is 75.89% for stock TSLA
For 260th day in future, prediction score is 73.88% for stock TSLA
For 270th day in future, prediction score is 74.06% for stock TSLA
For 280th day in future, prediction score is 75.06% for stock TSLA
For 290th day in future, prediction score is 73.39% for stock TSLA
For 300th day in future, prediction score is 73.47% for stock TSLA
For 310th day in future, prediction score is 70.9% for stock TSLA


Result:
Our model can predict with prediction score > 70% as shown below for each stock
PYPL: 531
CTSH: 432
TSLA: 312


In [16]:
print("Result:")
print("Our model can predict with prediction score above 70% for number of days as shown below for each stock")
for stock in result:
    print(stock+":",result[stock])

Result:
Our model can predict with prediction score above 70% for number of days as shown below for each stock
PYPL: 531
CTSH: 432
TSLA: 312
